# Backtesting Strategy

This notebook backtests the trained models.

In [ ]:
import pandas as pd
import sys
import os
import joblib
import matplotlib.pyplot as plt

# Add src to path
sys.path.append(os.path.abspath('../src'))
from backtest import Backtester
from features import FeatureEngineer

# Load data and model
data_path = '../data/features/AAPL_1h_features.parquet'
model_path = '../models/xgboost_baseline.joblib'

if os.path.exists(data_path) and os.path.exists(model_path):
    df = pd.read_parquet(data_path)
    model = joblib.load(model_path)
    
    # Prepare features
    feature_cols = [col for col in df.columns 
                   if col not in ['target_return', 'target_direction', 'target_binary', 
                                  'target_next_close', 'symbol', 'timestamp', 'datetime']]
    
    X = df[feature_cols]
    
    # Generate predictions
    print("Generating predictions...")
    probs = model.predict_proba(X)[:, 1]
    
    # Generate signals (Threshold strategy)
    threshold = 0.6
    signals = pd.Series(0, index=df.index)
    signals[probs > threshold] = 1  # Buy
    signals[probs < (1 - threshold)] = -1  # Sell
    
    # Run Backtest
    backtester = Backtester(initial_capital=10000)
    results, metrics, trades = backtester.run_backtest(df, signals)
    
    print("\nBacktest Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v}")
        
    # Plot
    fig = backtester.plot_results(results)
    plt.show()
    
    # Save results
    os.makedirs('../results', exist_ok=True)
    results.to_csv('../results/backtest_results.csv')
else:
    print("Data or model not found.")